In [2]:
from pathlib import Path
#import json
#import numpy as np
#import pandas as pd
# import scanpy as sc
# import gseapy as gp
# from tqdm import tqdm
# from gears import PertData, GEARS
# from pathlib import Path
# from Bio import Entrez, SeqIO
# import mygene
# import gzip
# from transformers import AutoTokenizer, AutoModel
# import torch

In [ ]:
data_dir = Path('/Users/djemec/data/jepa/v0_3')
protein_dir = data_dir / 'uniprot'

train_dir = data_dir / 'training'
pretrain_dir = data_dir / 'training'
splits= ['train','val','test']


pert_dir = data_dir / 'pert_embd'